In [2]:
import pandas as pd
import re
import json
import csv
from datetime import datetime

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
json_output_filepath = '/Users/ekim/workspace/personal/dd-bot/dev/json.csv'

In [4]:
def get_lst_of_mapped_orders():
    with open(json_output_filepath, 'r') as f:
        data = json.load(f)
        
    return data
    
    
data = get_lst_of_mapped_orders()
data

[{'Order': 'A348DFB6',
  'Delivered': 'The order was delivered at 11:47 PM on September 28, 2023.',
  'Pick Up Location': '801 Lafayette St, Laredo, TX 78041, USA',
  'Order Details': {'1 × Ice Small (Fountain, Frozen, & Tea)': '$4.99',
   ' 2 × TXB Water Bottle (24 pk) (16.9 oz) (Water TX)': '$14.98'},
  'Subtotal': '$19.97',
  'Subtotal Tax': '$0.41',
  'Commission (23%)': '-$4.59',
  'Total Customer Refund': '-$0.00',
  'Estimated Payout': '$15.79',
  'Transaction #8087670804 - Delivery': '$15.79'},
 {'Order': 'EC218FA4',
  'Delivered': 'The order was delivered at 11:22 PM on September 28, 2023.',
  'Pick Up Location': '800 W Interstate 20, Big Spring, TX 79720, USA',
  'Order Details': {'1 × Ice Small (Fountain, Frozen, & Tea)': '$4.99'},
  'Subtotal': '$4.99',
  'Subtotal Tax': '$0.41',
  'Commission (16%)': '-$0.80',
  'Total Customer Refund': '-$0.00',
  'Estimated Payout': '$4.60',
  'Transaction #8087614462 - Delivery': '$4.60'},
 {'Order': '7D285B12',
  'Delivered': 'The orde

In [5]:
def map_order_to_location(orders):
    order_id_to_pickup_location  = {}
    for order in orders:
        order_id = order['Order']
        store_addrs = order['Pick Up Location']
        order_id_to_pickup_location[order_id] = store_addrs
    return order_id_to_pickup_location

order_to_location = map_order_to_location(data)
order_to_location

{'A348DFB6': '801 Lafayette St, Laredo, TX 78041, USA',
 'EC218FA4': '800 W Interstate 20, Big Spring, TX 79720, USA',
 '7D285B12': '800 W Interstate 20, Big Spring, TX 79720, USA',
 'F9F46AEF': '1001 W Van Alstyne Pkwy, Van Alstyne, TX 75495, USA',
 '7D8C6B67': '3702 FM2147, Horseshoe Bay, TX 78657, USA',
 '702662D6': '201 S 9th St, Durant, OK 74701, USA',
 '00BEC8AB': '816 N Center St, Bonham, TX 75418, USA',
 'F20F6AE7': '801 Lafayette St, Laredo, TX 78041, USA',
 'CA9E35CB': '801 Lafayette St, Laredo, TX 78041, USA',
 'D5782A52': '2503 TX-349, Midland, TX 79706, USA',
 '5CDF188C': '3702 FM2147, Horseshoe Bay, TX 78657, USA',
 '0BC3FC5E': '2320 W Main St, Durant, OK 74701, USA'}

In [6]:
lst_of_tups = list(order_to_location.items())

df = pd.DataFrame(lst_of_tups, columns=['order_id', 'pickup_location'])
# Remove all commas
df['pickup_location'] = df['pickup_location'].str.replace(',', '')

# Remove "USA" substring
df['pickup_location'] = df['pickup_location'].str.replace('USA', '')


df['pickup_location']

0                 801 Lafayette St Laredo TX 78041 
1          800 W Interstate 20 Big Spring TX 79720 
2          800 W Interstate 20 Big Spring TX 79720 
3     1001 W Van Alstyne Pkwy Van Alstyne TX 75495 
4               3702 FM2147 Horseshoe Bay TX 78657 
5                     201 S 9th St Durant OK 74701 
6                  816 N Center St Bonham TX 75418 
7                 801 Lafayette St Laredo TX 78041 
8                 801 Lafayette St Laredo TX 78041 
9                     2503 TX-349 Midland TX 79706 
10              3702 FM2147 Horseshoe Bay TX 78657 
11                  2320 W Main St Durant OK 74701 
Name: pickup_location, dtype: object

In [8]:
# Regular expression to capture address, city, state, and zip_code
pattern = r'(?P<address>^.+?)\s(?P<city>\w+\s*\w*)\s(?P<state>[A-Z]{2})\s(?P<zip_code>\d{5})'

# Extract components into new DataFrame columns
df_extracted = df['pickup_location'].str.extract(pattern)
# Concatenating the new columns with the original DataFrame
pickup_location_df = pd.concat([df, df_extracted], axis=1)

street_num_pattern = r'(?P<street_num>\b\d{3,5}\b)'


addrs_extracted_df = pickup_location_df['address'].str.extract(street_num_pattern)



pickup_location_df = pd.concat([pickup_location_df, addrs_extracted_df], axis=1)
# pickup_location_df['zip_code'] = pickup_location_df['zip_code'].astype(str)

pickup_location_df

,order_id,pickup_location,address,city,state,zip_code,street_num
0,A348DFB6,801 Lafayette St Laredo TX 78041,801 Lafayette,St Laredo,TX,78041,801
1,EC218FA4,800 W Interstate 20 Big Spring TX 79720,800 W Interstate 20,Big Spring,TX,79720,800
2,7D285B12,800 W Interstate 20 Big Spring TX 79720,800 W Interstate 20,Big Spring,TX,79720,800
3,F9F46AEF,1001 W Van Alstyne Pkwy Van Alstyne TX 75495,1001 W Van Alstyne Pkwy,Van Alstyne,TX,75495,1001
4,7D8C6B67,3702 FM2147 Horseshoe Bay TX 78657,3702 FM2147,Horseshoe Bay,TX,78657,3702
5,702662D6,201 S 9th St Durant OK 74701,201 S 9th,St Durant,OK,74701,201
6,00BEC8AB,816 N Center St Bonham TX 75418,816 N Center,St Bonham,TX,75418,816
7,F20F6AE7,801 Lafayette St Laredo TX 78041,801 Lafayette,St Laredo,TX,78041,801
8,CA9E35CB,801 Lafayette St Laredo TX 78041,801 Lafayette,St Laredo,TX,78041,801
9,D5782A52,2503 TX-349 Midland TX 79706,2503 TX-349,Midland,TX,79706,2503


In [60]:
file_path = '/Users/ekim/workspace/personal/dd-bot/dev/store_list.xlsx'
# # print(file_path)

store_info_masterset_df = pd.read_excel(file_path, index_col=0)


street_num_pattern = r'(?P<street_num>\b\d{3,5}\b)'
street_num_df = store_info_masterset_df['Address'].str.extract(street_num_pattern)

store_info_masterset_df = pd.concat([store_info_masterset_df, street_num_df], axis=1)

store_info_masterset_df['Site #'] = store_info_masterset_df['Site #'].astype(str)
store_info_masterset_df['Zip'] = store_info_masterset_df['Zip'].astype(str)
store_info_masterset_df['street_num'] = store_info_masterset_df['street_num'].astype(str)

store_info_masterset_df.dtypes


Site #              object
Site Description    object
Address             object
City                object
County              object
State               object
Zip                 object
Notes               object
street_num          object
dtype: object

In [83]:
master_df = pd.merge(store_info_masterset_df, pickup_location_df, left_on=['Zip', 'street_num'], right_on=['zip_code', 'street_num'], how='inner')

In [84]:
master_df.drop(['Site Description', 'City', 'County', 'State', 'address', 'city', 'state', 'Zip', 'zip_code', 'Notes', 'street_num'], axis=1, inplace=True)
master_df

,Site #,Address,order_id,pickup_location
0,6,816 North Center,00BEC8AB,816 N Center St Bonham TX 75418
1,14,201 South 9th,702662D6,201 S 9th St Durant OK 74701
2,16,2320 West Main,0BC3FC5E,2320 W Main St Durant OK 74701
3,32,1001 Van Alstyne Parkway,F9F46AEF,1001 W Van Alstyne Pkwy Van Alstyne TX 75495
4,67,3702 West FM 2147,7D8C6B67,3702 FM2147 Horseshoe Bay TX 78657
5,67,3702 West FM 2147,5CDF188C,3702 FM2147 Horseshoe Bay TX 78657
6,70,800 IH 20 West,EC218FA4,800 W Interstate 20 Big Spring TX 79720
7,70,800 IH 20 West,7D285B12,800 W Interstate 20 Big Spring TX 79720
8,71,2503 Rankin Hwy,D5782A52,2503 TX-349 Midland TX 79706
9,80,801 West Lafayette,A348DFB6,801 Lafayette St Laredo TX 78041


In [85]:
master_df = master_df[['Site #', 'order_id', 'Address', 'pickup_location']]

In [86]:
master_df

,Site #,order_id,Address,pickup_location
0,6,00BEC8AB,816 North Center,816 N Center St Bonham TX 75418
1,14,702662D6,201 South 9th,201 S 9th St Durant OK 74701
2,16,0BC3FC5E,2320 West Main,2320 W Main St Durant OK 74701
3,32,F9F46AEF,1001 Van Alstyne Parkway,1001 W Van Alstyne Pkwy Van Alstyne TX 75495
4,67,7D8C6B67,3702 West FM 2147,3702 FM2147 Horseshoe Bay TX 78657
5,67,5CDF188C,3702 West FM 2147,3702 FM2147 Horseshoe Bay TX 78657
6,70,EC218FA4,800 IH 20 West,800 W Interstate 20 Big Spring TX 79720
7,70,7D285B12,800 IH 20 West,800 W Interstate 20 Big Spring TX 79720
8,71,D5782A52,2503 Rankin Hwy,2503 TX-349 Midland TX 79706
9,80,A348DFB6,801 West Lafayette,801 Lafayette St Laredo TX 78041


In [87]:
order_to_site_num_dict = master_df.set_index('order_id')['Site #'].to_dict()
order_to_site_num_dict

{'00BEC8AB': '6',
 '702662D6': '14',
 '0BC3FC5E': '16',
 'F9F46AEF': '32',
 '7D8C6B67': '67',
 '5CDF188C': '67',
 'EC218FA4': '70',
 '7D285B12': '70',
 'D5782A52': '71',
 'A348DFB6': '80',
 'F20F6AE7': '80',
 'CA9E35CB': '80'}